In [1]:
import sys

sys.path.append('D:\PythonProjects\\bk-monitor\\venv2\share\scripts')
import hello

当前运行环境缺少部分需要的环境变量，可能导致项目运行报错，请确认以下环境变量已经配置并赋值: BK_PAAS2_URL
注意，当前版本为社区版环境，请确认 BKPAAS_MAJOR_VERSION 已经配置并正确赋值


In [ ]:
"""
新增策略排查脚本，排查策略关联的通知组在业务下不存在的策略。(通知组就是告警组)

遍历全部策略， 查看策略配置的通知组id，是否在当前业务下存在。 
    -如果查询不到， 则去掉业务过滤条件查询， 
    -如果存在，则将通知组的名称作为查询条件，在当前业务下查询。（查询当前业务下相同名称的告警组）
    如果有记录， 则替换为当前查询到的通知组id。并打印。
    
替换功能可以通过一个配置项（是否预览）开启。
开启预览，则仅打印信息不替换。 不开启预览，则替换+打印信息。

策略id， 策略名称， 业务id， 关联的通知组id+对应的业务id，本业务下相同名称的通知组id 

关联的通知组，需要的是非当前业务下的告警组。
"""

In [ ]:
from bkmonitor.models.strategy import StrategyModel, UserGroup
from bkmonitor.models.fta.action import StrategyActionConfigRelation
from collections import defaultdict
from pprint import pprint


def create_user_group():
    """
    创建告警组
    """
    # 清空数据
    UserGroup.objects.all().delete()
    user_group_ids = []
    for i in range(1, 6):
        instance = UserGroup.objects.create(name=f"test_user_group_{i}", bk_biz_id=2, desc=f"test_user_group_{i}_desc")
        user_group_ids.append(instance.id)

    # 创建具有相同名称的告警组
    for i in range(1, 6):
        instance = UserGroup.objects.create(name=f"test_user_group_{i}", bk_biz_id=2, desc=f"test_user_group_{i}_desc")
        user_group_ids.append(instance.id)

    # 创建非当前业务下的告警组
    for i in range(1, 6):
        instance = UserGroup.objects.create(name=f"test_user_group_{i}", bk_biz_id=6, desc=f"test_user_group_{i}_desc")
        user_group_ids.append(instance.id)

    for i in range(1, 6):
        instance = UserGroup.objects.create(name=f"test_user_group_{i}", bk_biz_id=5, desc=f"test_user_group_{i}_desc")
        user_group_ids.append(instance.id)

    return user_group_ids


def create_strategy():
    """
    创建策略
    """
    # 清空数据  
    StrategyModel.objects.all().delete()

    strategy_ids = []
    for i in range(1, 21):
        instance = StrategyModel.objects.create(name=f"test_stragety_{i}", bk_biz_id=2)
        strategy_ids.append(instance.id)
    return strategy_ids


# 创建告警组
def create_strategy_action_config():
    StrategyActionConfigRelation.objects.all().delete()

    # 创建告警组
    user_group_ids = create_user_group()
    # 创建新的策略
    stragety_ids = create_strategy()
    for i in range(len(stragety_ids)):
        StrategyActionConfigRelation.objects.create(config_id=0, strategy_id=stragety_ids[i],
                                                    user_groups=user_group_ids[i:i + 2])

    # 打印关联信息
    query_set = StrategyActionConfigRelation.objects.all().values("id", "strategy_id", "user_groups")
    for item in query_set:
        user_groups = UserGroup.objects.filter(id__in=item["user_groups"]).values("id", "name", "bk_biz_id")
        strategy = StrategyModel.objects.get(id=item["strategy_id"])
        item["user_groups_info"] = user_groups
        item["strategy_name"] = strategy.name
        pprint(item)


def show_strategy_action_config():
    # 打印关联信息
    query_set = StrategyActionConfigRelation.objects.all().values("id", "strategy_id", "user_groups")
    for item in query_set:
        user_groups = UserGroup.objects.filter(id__in=item["user_groups"]).values("id", "name", "bk_biz_id")
        strategy = StrategyModel.objects.get(id=item["strategy_id"])
        item["user_groups_info"] = user_groups
        item["strategy_name"] = strategy.name
        pprint(item)


# show_strategy_action_config()
create_strategy_action_config()

In [11]:
from bkmonitor.management.commands.run_cmd import parse_usergroup_not_under_business 

parse_usergroup_not_under_business(True)

策略id，策略名称，业务id，关联的非本业务的告警组信息，本业务下相同名称的告警组ID
1744,test_stragety_10,2,{'id': 999, 'name': 'test_user_group_1', 'bk_biz_id': 6},[994, 989]
1745,test_stragety_11,2,{'id': 999, 'name': 'test_user_group_1', 'bk_biz_id': 6},[994, 989]
1745,test_stragety_11,2,{'id': 1000, 'name': 'test_user_group_2', 'bk_biz_id': 6},[995, 990]
1746,test_stragety_12,2,{'id': 1000, 'name': 'test_user_group_2', 'bk_biz_id': 6},[995, 990]
1746,test_stragety_12,2,{'id': 1001, 'name': 'test_user_group_3', 'bk_biz_id': 6},[996, 991]
1747,test_stragety_13,2,{'id': 1001, 'name': 'test_user_group_3', 'bk_biz_id': 6},[996, 991]
1747,test_stragety_13,2,{'id': 1002, 'name': 'test_user_group_4', 'bk_biz_id': 6},[997, 992]
1748,test_stragety_14,2,{'id': 1002, 'name': 'test_user_group_4', 'bk_biz_id': 6},[997, 992]
1748,test_stragety_14,2,{'id': 1003, 'name': 'test_user_group_5', 'bk_biz_id': 6},[998, 993]
1749,test_stragety_15,2,{'id': 1003, 'name': 'test_user_group_5', 'bk_biz_id': 6},[998, 993]
1749,test_stragety_15,2,{'id

In [12]:
parse_usergroup_not_under_business(False)

策略id，策略名称，业务id，替换后关联的告警组信息
1744,test_stragety_10,2,{'id': 998, 'name': 'test_user_group_5', 'bk_biz_id': 2}
1744,test_stragety_10,2,{'id': 994, 'name': 'test_user_group_1', 'bk_biz_id': 2}
1744,test_stragety_10,2,{'id': 989, 'name': 'test_user_group_1', 'bk_biz_id': 2}
1745,test_stragety_11,2,{'id': 994, 'name': 'test_user_group_1', 'bk_biz_id': 2}
1745,test_stragety_11,2,{'id': 989, 'name': 'test_user_group_1', 'bk_biz_id': 2}
1745,test_stragety_11,2,{'id': 995, 'name': 'test_user_group_2', 'bk_biz_id': 2}
1745,test_stragety_11,2,{'id': 990, 'name': 'test_user_group_2', 'bk_biz_id': 2}
1746,test_stragety_12,2,{'id': 995, 'name': 'test_user_group_2', 'bk_biz_id': 2}
1746,test_stragety_12,2,{'id': 990, 'name': 'test_user_group_2', 'bk_biz_id': 2}
1746,test_stragety_12,2,{'id': 996, 'name': 'test_user_group_3', 'bk_biz_id': 2}
1746,test_stragety_12,2,{'id': 991, 'name': 'test_user_group_3', 'bk_biz_id': 2}
1747,test_stragety_13,2,{'id': 996, 'name': 'test_user_group_3', 'bk_biz_id': 2}
1

In [ ]:
show_strategy_action_config()